In [2]:
from PIL import Image

def resize_faces(person_name):
    for i in range(10):
        img = Image.open(f"{person_name}{i}.png")
        img = img.resize((100, 100), Image.BILINEAR)
        path = person_name.split('/')[-1]
        # print(path)
        img.save(f"./resized/{path}{i}.png")

for person in ['A', 'B', 'C']:
    resize_faces(f"./images/face{person}")


In [3]:
import numpy as np
import cv2
import os

def find_average_face(person_name, num_images=10):
    sum_img = None
    valid_images = 0  # Counter for successfully loaded images
    for i in range(num_images):
        img_path = f"{person_name}{i}.png"
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Image at '{img_path}' could not be loaded.")
            continue
        img = img.astype(np.float32)  # Ensure proper type for arithmetic
        if sum_img is None:
            sum_img = img
        else:
            sum_img = cv2.add(sum_img, img)  # Use cv2.add to ensure proper handling of overflow
        valid_images += 1
    
    if valid_images > 0:
        # Division by 10 as per specific requirement
        avg_img = (sum_img / 10).astype(np.uint8)
        std_img_dir = './std_img/'
        if not os.path.exists(std_img_dir):
            os.makedirs(std_img_dir)
        cv2.imwrite(f"{std_img_dir}std{os.path.basename(person_name)}.png", avg_img)
        print(f"Average face for {person_name} saved.")
    else:
        print(f"No valid images processed for {person_name}.")

# Assuming your resized images are stored in a directory correctly formatted in the path
for person in ['A', 'B', 'C']:
    find_average_face(f"./resized/face{person}", 10)  # Adjust num_images if needed


Average face for ./resized/faceA saved.
Average face for ./resized/faceB saved.
Average face for ./resized/faceC saved.


In [4]:
import numpy as np
import cv2
import os

# Load standard faces
std_faces = {}
for person in ['A', 'B', 'C']:
    std_face = cv2.imread(f'./std_img/stdFace{person}.png', cv2.IMREAD_COLOR)
    if std_face is not None:
        std_faces[person] = cv2.cvtColor(std_face, cv2.COLOR_BGR2GRAY)
    else:
        print(f"Error loading standard face for person {person}")


In [1]:
def recognize_face_real_time(std_faces, face_size=(100, 100)):
    # Initialize the video capture object
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, face_size)

            min_error = np.inf
            match = None
            for person, std_face in std_faces.items():
                error = np.sum(cv2.absdiff(face_resized, std_face))
                if error < min_error:
                    min_error = error
                    match = person
            
            # Display the matching result on the frame
            cv2.putText(frame, f'Match: {match}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Display the resulting frame
        cv2.imshow('Frame', frame)

        # Break the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

recognize_face_real_time(std_faces)


NameError: name 'std_faces' is not defined

In [1]:
from PIL import Image
import numpy as np
import cv2
import os

class ImageProcessor:
    def __init__(self, person_names, base_dir='./', resized_dir='./resized/', std_dir='./std_img/'):
        self.person_names = person_names
        self.base_dir = base_dir
        self.resized_dir = resized_dir
        self.std_dir = std_dir
        if not os.path.exists(self.resized_dir):
            os.makedirs(self.resized_dir)
        if not os.path.exists(self.std_dir):
            os.makedirs(self.std_dir)

    def resize_faces(self):
        for person in self.person_names:
            for i in range(10):
                img_path = f"{self.base_dir}images/face{person}{i}.png"
                img = Image.open(img_path)
                img = img.resize((224, 224), Image.BILINEAR)
                img.save(f"{self.resized_dir}face{person}{i}.png")

    def find_average_face(self, num_images=10):
        for person in self.person_names:
            sum_img = None
            valid_images = 0
            for i in range(num_images):
                img_path = f"{self.resized_dir}face{person}{i}.png"
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Warning: Image at '{img_path}' could not be loaded.")
                    continue
                img = img.astype(np.float32)
                if sum_img is None:
                    sum_img = img
                else:
                    sum_img = cv2.add(sum_img, img)
                valid_images += 1
            
            if valid_images > 0:
                avg_img = (sum_img / valid_images).astype(np.uint8)
                cv2.imwrite(f"{self.std_dir}stdFace{person}.png", avg_img)
                print(f"Average face for {person} saved.")
            else:
                print(f"No valid images processed for {person}.")


In [2]:
class FaceRecognizer:
    def __init__(self, std_faces, face_size=(224, 224)):
        self.std_faces = std_faces
        self.face_size = face_size
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def load_standard_faces(self, std_dir):
        for person in self.std_faces.keys():
            std_face = cv2.imread(f'{std_dir}stdface{person}.png', cv2.IMREAD_COLOR)
            if std_face is not None:
                self.std_faces[person] = cv2.cvtColor(std_face, cv2.COLOR_BGR2GRAY)
            else:
                print(f"Error loading standard face for person {person}")

    def recognize_face_real_time(self):
        cap = cv2.VideoCapture(0)

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

            for (x, y, w, h) in faces:
                face = gray[y:y+h, x:x+w]
                face_resized = cv2.resize(face, self.face_size)
                min_error = np.inf
                match = None
                for person, std_face in self.std_faces.items():
                    error = np.sum(cv2.absdiff(face_resized, std_face))
                    if error < min_error:
                        min_error = error
                        match = person
                
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(frame, f'Match: {match}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

            cv2.imshow('Frame', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


In [3]:
#Driver Code
person_names = ['A', 'B', 'C']
std_faces = {name: None for name in person_names}

image_processor = ImageProcessor(person_names)
image_processor.resize_faces()
image_processor.find_average_face()

face_recognizer = FaceRecognizer(std_faces)
face_recognizer.load_standard_faces(image_processor.std_dir)
face_recognizer.recognize_face_real_time()


Average face for A saved.
Average face for B saved.
Average face for C saved.
